# Experiments

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path=".env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [4]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-3.5-turbo"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [3]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

c:\Users\INDRANEEL GHOSH\intro-to-langsmith\ls-academy\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-524b955c' at:
https://smith.langchain.com/o/f0f2baf4-9fbd-447b-a104-d11f741be5ea/datasets/579ae2d5-2340-47e3-8ea7-fd1809988b58/compare?selectedSessions=99d6b266-5f6e-4664-89f3-eab888444c6f




10it [00:33,  3.34s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support offline evaluation?,The provided context does not explicitly menti...,None,"Yes, LangSmith supports offline evaluation thr...",1,4.503890,050324f4-6eaf-4e01-9522-ea5889c274e8,1ed28f0b-5c64-4eea-8793-90a4751b0513
1,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Lang...,None,To trace with the @traceable decorator in Pyth...,1,3.734890,39b7204c-eee1-424b-bb66-f7781f588544,f05710d8-0550-40aa-a844-709ac4fe32f7
2,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.670138,64cf94d5-f60c-4c0e-bbf3-67866fb2c300,9962c13c-eb87-4e7c-8523-cd61fd8af85a
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,1,5.404187,7e0f0fc5-5b7a-4fe1-9ca9-fadeca449936,37d8729f-d1f4-43f8-b7c2-eb4794c37229
4,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. It ...",None,"Yes, LangSmith supports online evaluation as a...",1,2.546234,807404c2-f9e4-46f2-a44a-07401b2e432e,97e19000-9b42-46d1-a4c4-0cef2ee75a55
5,How do I pass metadata in with @traceable?,"To pass metadata with @traceable in LangSmith,...",None,You can pass metadata with the @traceable deco...,1,2.964342,8fe4b102-e2c6-4c9b-8826-d6d6aadb12c4,cf7818fa-669d-479a-a371-9954d05e5348
6,What testing capabilities does LangSmith have?,LangSmith allows you to run multiple experimen...,None,LangSmith offers capabilities for creating dat...,1,3.496592,9ce77167-0db2-451d-b6d0-a4b4e40adfb3,d6aeff3a-62e4-4f37-88dc-df456e034c61
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.010984,a4ae1232-4a84-48de-b5b6-eff9572cc1ac,f25ef29d-339c-461f-a86d-34ad1240d7e1
8,Can LangSmith be used for finetuning and model...,"No, LangSmith is focused on LLM observability ...",None,"Yes, LangSmith can be used for fine-tuning and...",1,3.066928,b228d819-0dd8-4977-b754-149e1fe21aec,9a6d73e5-104e-447f-8ad6-a828bc9acac4
9,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.335854,f6f68b1c-2752-458e-af0d-722605500e0b,927903a7-398c-489c-a218-fc2ecf5b4c0f


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [5]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-9c5bb723' at:
https://smith.langchain.com/o/f0f2baf4-9fbd-447b-a104-d11f741be5ea/datasets/579ae2d5-2340-47e3-8ea7-fd1809988b58/compare?selectedSessions=017231c7-84f0-44e2-9a02-b96f4b23e818




10it [00:22,  2.22s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support offline evaluation?,LangSmith currently supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.049592,050324f4-6eaf-4e01-9522-ea5889c274e8,6a30f000-82ae-459e-ab35-041ad1f4d85e
1,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,2.545122,39b7204c-eee1-424b-bb66-f7781f588544,1bbb1c5a-ae21-4cc6-ab96-df27a7091e86
2,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.840944,64cf94d5-f60c-4c0e-bbf3-67866fb2c300,9dffe613-d5e4-4e6e-890d-24ed47853db7
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith while using Lan...,None,To set up tracing to LangSmith while using Lan...,0,2.338107,7e0f0fc5-5b7a-4fe1-9ca9-fadeca449936,e44b2f8d-33cd-4976-9af1-5eff0e2f1b31
4,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluations. Th...",None,"Yes, LangSmith supports online evaluation as a...",1,1.832325,807404c2-f9e4-46f2-a44a-07401b2e432e,cb0af33d-dedf-48ec-9bcb-ff73ef198a65
5,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can in...",None,You can pass metadata with the @traceable deco...,1,1.932171,8fe4b102-e2c6-4c9b-8826-d6d6aadb12c4,b3ad18bb-e760-4c20-aeaa-c5f903ccdaa7
6,What testing capabilities does LangSmith have?,LangSmith in LangChain offers the capability t...,None,LangSmith offers capabilities for creating dat...,1,2.069611,9ce77167-0db2-451d-b6d0-a4b4e40adfb3,fe820d6c-d708-4564-9f1c-4e9a03fe6ba5
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.929206,a4ae1232-4a84-48de-b5b6-eff9572cc1ac,fd242c7d-1b19-411a-b2f8-62ee9701d973
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for observabil...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.764259,b228d819-0dd8-4977-b754-149e1fe21aec,ed77a190-1fab-463c-a8c1-cd6447293373
9,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.270180,f6f68b1c-2752-458e-af0d-722605500e0b,ddfa140c-034c-491c-a838-4ab8c894fc53


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [11]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "050324f4-6eaf-4e01-9522-ea5889c274e8",
            "39b7204c-eee1-424b-bb66-f7781f588544"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-d326fb76' at:
https://smith.langchain.com/o/f0f2baf4-9fbd-447b-a104-d11f741be5ea/datasets/579ae2d5-2340-47e3-8ea7-fd1809988b58/compare?selectedSessions=45abeddc-1dab-44cb-9c31-c5831ed41ba2




2it [00:04,  2.20s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support offline evaluation?,LangSmith focuses on online evaluation techniq...,None,"Yes, LangSmith supports offline evaluation thr...",1,1.621759,050324f4-6eaf-4e01-9522-ea5889c274e8,e4b6ba73-c01f-4efa-ae1e-48edb4b5595e
1,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.968915,39b7204c-eee1-424b-bb66-f7781f588544,3bc13de4-b9a9-43d8-9810-402a0a234dfc


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [10]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-cbba406d' at:
https://smith.langchain.com/o/f0f2baf4-9fbd-447b-a104-d11f741be5ea/datasets/579ae2d5-2340-47e3-8ea7-fd1809988b58/compare?selectedSessions=24c17cf6-4b9e-41df-8ee2-512dc3e6586b




20it [00:41,  2.09s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support offline evaluation?,"LangSmith does not support offline evaluation,...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.954868,050324f4-6eaf-4e01-9522-ea5889c274e8,6183573c-ac38-4256-9e66-768dca838b9e
1,How can I trace with the @traceable decorator?,To trace with the @traceable decorator in Pyth...,None,To trace with the @traceable decorator in Pyth...,1,1.925185,39b7204c-eee1-424b-bb66-f7781f588544,036c8f31-dc9f-4956-a29b-bf57c6d0870f
2,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents,...",None,"Yes, LangSmith can be used to evaluate agents....",0,2.157454,64cf94d5-f60c-4c0e-bbf3-67866fb2c300,889d88c3-9e07-432b-baad-1a77399aad1d
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,1,1.418641,7e0f0fc5-5b7a-4fe1-9ca9-fadeca449936,1492be32-d011-4b5e-aa11-85f7b64e38eb
4,Does LangSmith support online evaluation?,LangSmith supports online evaluation through L...,None,"Yes, LangSmith supports online evaluation as a...",0,6.614753,807404c2-f9e4-46f2-a44a-07401b2e432e,4663b3f2-7961-47a4-95bb-5545f165a34d
5,How do I pass metadata in with @traceable?,"To pass metadata using `@traceable`, you can i...",None,You can pass metadata with the @traceable deco...,1,1.472511,8fe4b102-e2c6-4c9b-8826-d6d6aadb12c4,c2d0b381-9f8a-4f08-b54b-1145049e1b0d
6,What testing capabilities does LangSmith have?,LangSmith in LangChain offers testing capabili...,None,LangSmith offers capabilities for creating dat...,1,1.557125,9ce77167-0db2-451d-b6d0-a4b4e40adfb3,2a610ed9-8d07-4d02-9830-39234acce261
7,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.498646,a4ae1232-4a84-48de-b5b6-eff9572cc1ac,7da9cc96-6089-45ae-8d6f-47b28a453de7
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.024658,b228d819-0dd8-4977-b754-149e1fe21aec,07727cac-fe85-483f-881a-5263c000cad1
9,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.340357,f6f68b1c-2752-458e-af0d-722605500e0b,abf5b49b-1489-40c6-ae53-ba6242ea482b


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [9]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-79bff23a' at:
https://smith.langchain.com/o/f0f2baf4-9fbd-447b-a104-d11f741be5ea/datasets/579ae2d5-2340-47e3-8ea7-fd1809988b58/compare?selectedSessions=b5a53d50-db82-428a-bf2e-04747bb5ce7f




10it [00:07,  1.34it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents ...",None,"Yes, LangSmith can be used to evaluate agents....",1,1.844709,64cf94d5-f60c-4c0e-bbf3-67866fb2c300,d20e142d-4020-410f-81c2-47b851b1f354
1,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,2.247007,39b7204c-eee1-424b-bb66-f7781f588544,3c80f02d-0939-4c30-8870-bce6ae24de98
2,Does LangSmith support offline evaluation?,LangSmith does not support offline evaluation ...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.866729,050324f4-6eaf-4e01-9522-ea5889c274e8,281ce80a-f892-4aaf-86f1-d3762ce8bb3d
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,1.526683,7e0f0fc5-5b7a-4fe1-9ca9-fadeca449936,98c6943b-cb36-42ff-b657-9eeec2527476
4,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation thro...",None,"Yes, LangSmith supports online evaluation as a...",0,2.150736,807404c2-f9e4-46f2-a44a-07401b2e432e,d21dcbf1-7956-4211-a538-0588720260f2
5,How do I pass metadata in with @traceable?,"To pass metadata with @traceable, you can incl...",None,You can pass metadata with the @traceable deco...,1,1.834512,8fe4b102-e2c6-4c9b-8826-d6d6aadb12c4,391e3dc4-eae7-41c7-8b83-2cb1e674e903
6,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.581912,9ce77167-0db2-451d-b6d0-a4b4e40adfb3,0ca7a327-ed0c-488f-b5b9-ea5177df2f50
7,How do I create user feedback with the LangSmi...,To create user feedback using the LangSmith SD...,None,To create user feedback with the LangSmith SDK...,1,1.816016,a4ae1232-4a84-48de-b5b6-eff9572cc1ac,02f57596-89b7-4eeb-9e54-ad8ce3e3ec8c
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on observabilit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.742525,b228d819-0dd8-4977-b754-149e1fe21aec,7a846a8e-7827-4669-8cca-ac305535df2e
9,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,1.984606,f6f68b1c-2752-458e-af0d-722605500e0b,7bd1e25c-9dec-4883-9dce-449041550cf1


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [8]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-c74385fa' at:
https://smith.langchain.com/o/f0f2baf4-9fbd-447b-a104-d11f741be5ea/datasets/579ae2d5-2340-47e3-8ea7-fd1809988b58/compare?selectedSessions=c5c23b1c-1b63-40f7-8800-994d8ddda3fe




10it [00:21,  2.16s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,Does LangSmith support offline evaluation?,LangSmith primarily supports online evaluation...,None,"Yes, LangSmith supports offline evaluation thr...",1,2.356743,050324f4-6eaf-4e01-9522-ea5889c274e8,49fb7d59-22d6-43d3-8cf2-f1778e7e79ec
1,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you ne...",None,To trace with the @traceable decorator in Pyth...,1,1.716632,39b7204c-eee1-424b-bb66-f7781f588544,8f05b79e-80d8-44ca-ad96-821d9ed69a9c
2,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,2.168937,64cf94d5-f60c-4c0e-bbf3-67866fb2c300,5368db68-231d-4fee-8956-be8dedc387b0
3,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.664076,7e0f0fc5-5b7a-4fe1-9ca9-fadeca449936,e9da5900-7940-43ad-92fb-f3c421ae3c0d
4,Does LangSmith support online evaluation?,LangSmith supports online evaluations for real...,None,"Yes, LangSmith supports online evaluation as a...",0,2.241130,807404c2-f9e4-46f2-a44a-07401b2e432e,106958e3-10dc-4b08-aaca-f594964216a5
5,How do I pass metadata in with @traceable?,"To pass metadata in with @traceable, you can u...",None,You can pass metadata with the @traceable deco...,1,2.055365,8fe4b102-e2c6-4c9b-8826-d6d6aadb12c4,396c7336-79b9-40dc-a803-fe241bc5e1f4
6,What testing capabilities does LangSmith have?,LangSmith allows users to run multiple experim...,None,LangSmith offers capabilities for creating dat...,1,1.625247,9ce77167-0db2-451d-b6d0-a4b4e40adfb3,3b93be36-a0c2-424e-ba28-8348473416e3
7,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.122264,a4ae1232-4a84-48de-b5b6-eff9572cc1ac,d4dbf98a-128b-4b04-b2c2-691065de1520
8,Can LangSmith be used for finetuning and model...,LangSmith is primarily designed for monitoring...,None,"Yes, LangSmith can be used for fine-tuning and...",1,1.837487,b228d819-0dd8-4977-b754-149e1fe21aec,ed5a8642-9861-44d3-ab57-dd51186646a6
9,What is LangSmith used for in three sentences?,LangSmith is a platform designed for building ...,None,LangSmith is a platform designed for the devel...,1,2.270074,f6f68b1c-2752-458e-af0d-722605500e0b,94d29c59-142e-475e-bfd5-5e8549841b11
